In [1]:
import itertools
import os

from py_alf import ALF_source, Simulation
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from tqdm.notebook import tqdm
from IPython.utils import io

In [3]:
alf_src = ALF_source()

In [4]:
def print_nested_dict(d, nesting=0, sep="  "):
    for k, v in d.items():
        print(nesting*sep + f"{k}: ", end="")
        if isinstance(v, dict):
            print("[dict]")
            print_nested_dict(v, nesting=nesting+1, sep=sep)
        else:
            print(f"{v}")
        if not nesting:
            print("")

In [5]:
print_nested_dict(alf_src.get_default_params("Hubbard"))

VAR_lattice: [dict]
  Model: [dict]
    comment: Value not relevant
    defined_in_base: False
    value: Hubbard
  Lattice_type: [dict]
    comment: 
    defined_in_base: False
    value: Square
  L1: [dict]
    comment: Length in direction a_1
    defined_in_base: False
    value: 6
  L2: [dict]
    comment: Length in direction a_2
    defined_in_base: False
    value: 6

VAR_Model_Generic: [dict]
  N_SUN: [dict]
    comment: Number of colors
    defined_in_base: True
    value: 2
  N_FL: [dict]
    comment: Number of flavors
    defined_in_base: True
    value: 1
  Phi_X: [dict]
    comment: Twist along the L_1 direction, in units of the flux quanta
    defined_in_base: False
    value: 0.0
  Phi_Y: [dict]
    comment: Twist along the L_2 direction, in units of the flux quanta
    defined_in_base: False
    value: 0.0
  Bulk: [dict]
    comment: Twist as a vector potential (.T.), or at the boundary (.F.)
    defined_in_base: False
    value: True
  N_Phi: [dict]
    comment: Total n

Hamiltonian for $N$ flavors:
$$
    H = [\mathrm{hop}] + \sum_i \frac U N \left(\sum_\sigma \left[n_{i\sigma} - \frac 1 2 \right]\right)^2 - \mu \sum_{i,\sigma} n_{i\sigma}
$$

In [6]:
# for the Hamiltonian see eq. (189) in 
Lx = Ly = 2
Nf = 4
U = -4.0
t = 0.1
#mu = 0.0
rescale_U = True
#mu = 0
use_Mz = False

projector = False
theta = 20

dtau = 0.04
beta = 40.0

NSUN = Nf
NFL = 1

if rescale_U:
    U *= Nf / 2

In [7]:
# print(os.getenv("ALF_DIR"))
# Simulation(
#     alf_src,
#     "Hubbard",
#     {},
#     machine="GNU"
# ).compile()

In [9]:
COMPILED = False

N_REFINEMENTS = 1

Dtaus = {
        1: [0.2],
        2: [0.1],
        3: [0.05],
        4: [0.01]
     }

Betas = {
        1: [10],
        2: [20],
        3: [40]
     }

Ts = {
      1: [0.01, 0.1],
      2: [1.0, 2.0],
     }
Us = {
      1: [4.0, 2.0, 1.0],
      2: [0.5, 0.1, -0.1, -0.5],
      3: [-1.0, -2.0, -4.0],
     }
Mus = np.linspace(-4,4,41)

## small version for testing
# Ts = {"t": [0.1],
#       "r": [1.0]}
# Us = {"x": [-1.0],
#       "y": [-2.0]}
# Mus = [0]


param_keys = ["t", "U", "mu", "beta", "dtau"]
observables = ["N", "E", "T", "V"]
obs_errs = ["d"+o for o in observables]
obs_signs = ["s"+o for o in observables]
obs_names = ["Part", "Ener", "Kin", "Pot"]

for curr_t, Tc in Ts.items():
    for curr_U, Uc in Us.items():
        for curr_dtau, DTc in Dtaus.items():
            for curr_beta, Bc in Betas.items():
        
                res = {k: [] for k in param_keys + observables + obs_errs + obs_signs}

                tot = len(Tc)*len(Uc)*len(DTc)*len(Bc)*len(Mus)
                desc=f"TUDB:{curr_t}{curr_U}{curr_dtau}{curr_beta}"

                for t, U, dtau, beta, mu in tqdm(itertools.product(Tc, Uc, DTc, Bc, Mus), total=tot,desc=desc):
                    with io.capture_output() as foo:
                        sim = Simulation(
                            alf_src,
                            "Hubbard",
                            {
                                "Lattice_type": "Square",
                                "L1": Lx,
                                "L2": Ly,
                                "ham_u": U,
                                "ham_chem": mu,
                                "ham_T": t,
                                "Projector": projector,
                                "Theta": theta,
                                "N_SUN": NSUN,
                                "N_FL": NFL,
                                "Mz": use_Mz,
                                "Dtau": dtau,
                                "beta": beta,
                            },
                            machine="GNU"
                        )

                        sim.run()

                        sim.analysis()
                        obs = sim.get_obs().iloc[0]

                        for o, do, so, o_name in zip(observables, obs_errs, obs_signs, obs_names):
                            res[o].append(obs[o_name + "_scal0"])
                            res[do].append(obs[o_name + "_scal0_err"])
                            res[so].append(obs[o_name + "_scal_sign"])
                        res["t"].append(t)
                        res["U"].append(U)
                        res["mu"].append(mu)
                        res["dtau"].append(dtau)
                        res["beta"].append(beta)


        res = pd.DataFrame(res)
        res.to_pickle(f"obs/L2x2_mu_data_T{curr_t}_U{curr_U}_dt{curr_dtau}_b{curr_beta}.pkl")

TUDB:1111:   0%|          | 0/246 [00:00<?, ?it/s]

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

...

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial config

KeyboardInterrupt: 

In [ ]:
# res[['mu', 'N']]

In [ ]:
# Ns = res["N"]
# dNs = res["dN"]
# Nmask = (abs(Ns) < Lx*Ly*Nf+1)
# Ns_sanitized = Ns[Nmask]
# MU_Ns = res["mu"][Nmask]
# mcenter_Ns = (abs(MU_Ns) <= 0.6)
# # plt.errorbar(x=MU_Ns[mcenter_Ns], y=Ns_sanitized[mcenter_Ns], yerr=dNs[Nmask][mcenter_Ns])
# plt.errorbar(x=MU_Ns, y=Ns_sanitized, yerr=dNs[Nmask], linewidth=0, marker='+')
# plt.show()

In [ ]:
# save_name = f"U{U}_t{t}"
# if projector:
#     save_name += "pth{theta}"
# if rescale_U:
#     save_name += "rescaledU"
# save_name += "_NEKV.npy"

# print("saving to", save_name)
# np.save(save_name, np.vstack((MU,Ns,Es)))